In [2]:
import os
import numpy as np
import pandas as pd

In [22]:
#Import CTD bottle nitrate data for calibration of SUNA

infile = './nitratewatersamples.txt'
NO3_bottle = pd.read_csv(infile, sep='\t', names = ["StationNr", "Depth", "Nitrate"])

AlignedFile = '%s.txt' % ('NO3_SUNA_Sample_aligned')
AlignedF = open(AlignedFile,'w')

Aligned = pd.DataFrame(columns=['Station','Depth','Nitrate','NitrateBinned'])

In [26]:
#Loop through SUNA files
for f_name in os.listdir('./SUNA_profiles'):
    if f_name.endswith('.txt'):
        f_name_elements = f_name.split('sta')
        #print(f_name_elements)
        f_name_getnr = f_name_elements[1].split('.txt')
        infile = './SUNA_profiles/%s' % (f_name)
        SUNA_profile = pd.read_csv(infile, sep=' ')
        #From bin ctd code from Corday
        inc = 1 # define depth increment to bin by (m)
        depthlist = np.arange(0,np.round(np.max(SUNA_profile.Depth), decimals = 0) + inc, inc)
        df = pd.DataFrame(columns=['Station','Depth','NitrateBinned'])
        for j in depthlist:
            #Modify following to read in all profiles
            #loc = df[(df.Cast == i) & (np.round(df.Depth_m, decimals = 0) == j)]
            loc = SUNA_profile[(np.round(SUNA_profile.Depth, decimals = 0) == j)]
            Nitratebinned = np.mean((loc["Nitrate(umol/L)"]))
            #Adjust "5" when looping through files
           # OutputString = "5,%i,%f\n" % (j,Nitratebinned)
            df = df.append({'Station': int(f_name_getnr[0]),'Depth':j,'NitrateBinned':Nitratebinned}, ignore_index=True)
    df.to_csv('%s%s%s' % (f_name_elements[0],'binned_',f_name_elements[1]), header=None, index=None, sep='\t', mode='a')
    
    #Find corresponding depths between files
    for index, row in NO3_bottle.iterrows():
        for indexSUNA, rowSUNA in df.iterrows():
            if int(row['StationNr']) == int(rowSUNA['Station']) and int(row['Depth']) == int(rowSUNA['Depth']):
                Aligned = Aligned.append({'Station': (int(rowSUNA['Station'])),'Depth':row['Depth'],'Nitrate' : row['Nitrate'],'NitrateBinned':rowSUNA['NitrateBinned']}, ignore_index=True)
                

In [27]:
Aligned

,Station,Depth,Nitrate,NitrateBinned
0,5.0,1.0,0.2471,NaN
1,5.0,30.0,0.5347,2.780000
2,5.0,150.0,14.1829,NaN
3,5.0,200.0,13.1632,15.480000
4,75.0,3.0,0.0861,NaN
...,...,...,...,...
57,87.0,4.0,0.0826,4.044500
58,87.0,13.0,0.1031,4.176667
59,87.0,28.0,0.1892,6.270000
60,87.0,75.0,7.0470,17.483333


In [28]:
Aligned.to_csv('Aligned_BinnedSUNA_bottleSamples.csv', header=None, index=None, sep='\t', mode='a')


In [ ]:
#Skip below, just saved for reference

In [17]:
#Find corresponding depths between files
for index, row in NO3_bottle.iterrows():
    for indexSUNA, rowSUNA in df.iterrows():
        if int(row['StationNr']) == int(rowSUNA['Station']) and int(row['Depth']) == int(rowSUNA['Depth']):
            print(row['Depth'],' ', row['Nitrate'],' ', rowSUNA['NitrateBinned'])

1.0   0.2471   nan
30.0   0.5347   2.78
150.0   14.1829   nan
200.0   13.1632   15.48


In [ ]:
#Above depth of 150 is returning a nan for the binned nitrate

#Raw unbinned SUNA nitrate data around depth of 150:

#147.515478 15.020000 
#149.549457 NaN 
#151.236311 15.880000 